# 🐱🐶 Oxford-IIIT Pet Classification - Quick Demo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YOUR_USERNAME/oxford-pets-classification/blob/main/notebooks/00_quick_demo.ipynb)

This notebook demonstrates the complete pipeline for binary pet classification (Cat vs Dog) using custom CNN architectures.

**What you'll see:**
- 🚀 Automatic setup and installation
- 📊 Training a CNN from scratch
- 📈 Visualizing results with interactive plots
- 🔍 Model interpretation with Grad-CAM

**Estimated time:** 10-15 minutes (with Colab GPU)

## ⚙️ Setup

First, let's clone the repository and install dependencies.

In [ ]:
# Clone repository (replace YOUR_USERNAME with your GitHub username)
!git clone https://github.com/YOUR_USERNAME/oxford-pets-classification.git
%cd oxford-pets-classification

In [ ]:
# Install dependencies
!pip install -q -r requirements.txt

print("✅ Installation complete!")

In [ ]:
# Check GPU availability
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️  Using device: {device}")

if device == "cuda":
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️  No GPU available. Training will be slower.")
    print("   Go to Runtime → Change runtime type → GPU")

## 📦 Imports

Import our custom modules.

In [ ]:
import sys
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, HTML

# Add project to path
sys.path.insert(0, '.')

from configs.config import BinaryClassificationConfig
from models.architectures import get_model, count_parameters
from utils.data_utils import prepare_binary_dataloaders
from utils.trainer import BinaryTrainer
from utils.visualization import plot_training_curves

import torch
import torch.nn as nn
import torch.optim as optim

print("✅ Imports successful!")

## 🎛️ Configuration

Set up training parameters. We'll use model **v3** (best performance) with reduced epochs for demo.

In [ ]:
# Configuration
config = BinaryClassificationConfig
config.MODEL_VERSION = 'v3'  # Best model
config.EPOCHS = 5  # Quick demo (use 20 for full training)
config.BATCH_SIZE = 64  # Larger batch for GPU
config.DEVICE = device
config.create_directories()

print("📋 Configuration:")
print(f"   Model: v3 (VGG-like with augmentation)")
print(f"   Epochs: {config.EPOCHS}")
print(f"   Batch Size: {config.BATCH_SIZE}")
print(f"   Learning Rate: {config.LEARNING_RATE}")
print(f"   Device: {config.DEVICE}")

## 📊 Load Dataset

The Oxford-IIIT Pet dataset will be downloaded automatically (~800MB, one-time download).

In [ ]:
print("📥 Loading dataset...")
print("   This may take a few minutes on first run.")

train_loader, val_loader, test_loader, id_to_binary = prepare_binary_dataloaders(
    config, 
    use_augmentation=True
)

print(f"\n✅ Dataset loaded!")
print(f"   Train samples: {len(train_loader.dataset):,}")
print(f"   Validation samples: {len(val_loader.dataset):,}")
print(f"   Test samples: {len(test_loader.dataset):,}")

### 🖼️ Visualize Sample Images

In [ ]:
# Get a batch of images
images, labels = next(iter(train_loader))

# Denormalize for visualization
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

fig, axes = plt.subplots(2, 4, figsize=(12, 6))
axes = axes.flatten()

for i in range(8):
    img = images[i].numpy().transpose(1, 2, 0)
    img = std * img + mean
    img = np.clip(img, 0, 1)
    
    label = "Dog" if labels[i].item() == 1 else "Cat"
    color = 'orange' if label == "Dog" else 'blue'
    
    axes[i].imshow(img)
    axes[i].set_title(label, color=color, fontweight='bold', fontsize=12)
    axes[i].axis('off')

plt.suptitle('Sample Training Images with Data Augmentation', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 🧠 Create Model

Model v3 is a VGG-like architecture with:
- 5 convolutional blocks
- Batch normalization
- Dropout regularization
- ~15M parameters

In [ ]:
# Create model
model = get_model('binary_v3')
model = model.to(config.DEVICE)

# Count parameters
total_params, trainable_params = count_parameters(model)

print("🧠 Model Architecture: BinaryCNN_v3")
print(f"   Total parameters: {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")
print(f"   Model size: ~{total_params * 4 / 1e6:.1f} MB")

## 🚀 Training

Let's train the model! This will take approximately:
- **With GPU:** ~2-3 minutes per epoch
- **Without GPU:** ~10-15 minutes per epoch

You'll see:
- Progress bar for each epoch
- Train and validation metrics
- Best model is saved automatically

In [ ]:
# Setup training
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(
    model.parameters(),
    lr=config.LEARNING_RATE,
    weight_decay=config.WEIGHT_DECAY
)

# Create trainer
save_dir = config.CHECKPOINT_DIR / config.EXPERIMENT_NAME
trainer = BinaryTrainer(
    model=model,
    device=config.DEVICE,
    criterion=criterion,
    optimizer=optimizer,
    save_dir=save_dir
)

print("✅ Training setup complete!")
print(f"   Checkpoints will be saved to: {save_dir}")

In [ ]:
# Train!
print("🚀 Starting training...\n")

history = trainer.fit(
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=config.EPOCHS,
    verbose=True
)

print("\n🎉 Training complete!")

## 📈 Training Results

Let's visualize the training progress.

In [ ]:
# Plot training curves
plot_training_curves(
    history,
    title="Binary Classification - Model v3"
)

# Print summary
print("\n📊 Training Summary:")
print(f"   Best Validation Accuracy: {trainer.best_val_acc:.4f} ({trainer.best_val_acc*100:.2f}%)")
print(f"   Best Epoch: {trainer.best_epoch}/{config.EPOCHS}")
print(f"   Final Train Accuracy: {history['train_acc'][-1]:.4f}")
print(f"   Final Val Accuracy: {history['val_acc'][-1]:.4f}")

## 🧪 Test Set Evaluation

Now let's see how well the model performs on unseen test data.

In [ ]:
print("🧪 Evaluating on test set...\n")

test_loss, test_acc = trainer.evaluate(test_loader)

print("="*70)
print("TEST RESULTS")
print("="*70)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
print("="*70)

# Compare with validation
print(f"\n📊 Comparison:")
print(f"   Best Validation Acc: {trainer.best_val_acc*100:.2f}%")
print(f"   Test Acc: {test_acc*100:.2f}%")
gap = abs(trainer.best_val_acc - test_acc) * 100
print(f"   Gap: {gap:.2f}%")

if gap < 2:
    print("   ✅ Good generalization!")
elif gap < 5:
    print("   ⚠️ Some overfitting detected")
else:
    print("   ❌ Significant overfitting")

## 🔮 Sample Predictions

Let's visualize some predictions on the test set.

In [ ]:
import random

# Get random test samples
model.eval()
indices = random.sample(range(len(test_loader.dataset)), 12)

fig, axes = plt.subplots(3, 4, figsize=(14, 10))
axes = axes.flatten()

with torch.no_grad():
    for idx, test_idx in enumerate(indices):
        img_tensor, true_label = test_loader.dataset[test_idx]
        
        # Predict
        img_input = img_tensor.unsqueeze(0).to(config.DEVICE)
        output = model(img_input)
        prob = torch.sigmoid(output).item()
        pred_label = 1 if prob > 0.5 else 0
        
        # Denormalize
        img = img_tensor.cpu().numpy().transpose(1, 2, 0)
        img = std * img + mean
        img = np.clip(img, 0, 1)
        
        # Plot
        axes[idx].imshow(img)
        axes[idx].axis('off')
        
        true_class = "Dog" if true_label == 1 else "Cat"
        pred_class = "Dog" if pred_label == 1 else "Cat"
        confidence = prob if pred_label == 1 else (1 - prob)
        
        title = f"True: {true_class}\nPred: {pred_class} ({confidence*100:.1f}%)"
        color = 'green' if true_label == pred_label else 'red'
        axes[idx].set_title(title, color=color, fontsize=10, fontweight='bold')

plt.suptitle('Sample Predictions on Test Set', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Calculate accuracy for these samples
correct = sum(1 for i in range(len(indices)) 
              if test_loader.dataset[indices[i]][1] == 
              (1 if torch.sigmoid(model(test_loader.dataset[indices[i]][0].unsqueeze(0).to(config.DEVICE))).item() > 0.5 else 0))
print(f"\n✅ Accuracy on displayed samples: {correct}/{len(indices)} ({correct/len(indices)*100:.1f}%)")

## 🎯 Next Steps

Great job! You've successfully trained a CNN for binary classification.

**Try these next:**

1. **Train longer:** Change `config.EPOCHS = 20` for better accuracy
2. **Compare models:** Try v0, v1, v2 to see the improvement
3. **Multi-class classification:** Check out `01_multiclass_classification.ipynb`
4. **Transfer learning:** See `02_transfer_learning.ipynb` for ResNet50

**Expected accuracies:**
- Model v0 (simple): ~85%
- Model v1: ~88%
- Model v2: ~91%
- Model v3 (this one, 20 epochs): **~94%**
- ResNet50 transfer learning: **~89%** (multi-class)

---

### 📚 Resources

- [GitHub Repository](https://github.com/YOUR_USERNAME/oxford-pets-classification)
- [Oxford-IIIT Pet Dataset](https://www.robots.ox.ac.uk/~vgg/data/pets/)
- [PyTorch Documentation](https://pytorch.org/docs/)

---

**Made with ❤️ for deep learning education**